In [7]:
import cv2 as cv
from math import atan2
import numpy as np
import os
 
def getOrientation(pts, img):
  # Construct a buffer used by the pca analysis
  sz = len(pts)
  data_pts = np.empty((sz, 2), dtype=np.float64)
  for i in range(data_pts.shape[0]):
    data_pts[i,0] = pts[i,0,0]
    data_pts[i,1] = pts[i,0,1]
 
  # Perform PCA analysis
  mean = np.empty((0))
  mean, eigenvectors, eigenvalues = cv.PCACompute2(data_pts, mean)
 
  # Store the center of the object
  cntr = (int(mean[0,0]), int(mean[0,1]))
 
  angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
 
  return angle

def rotate_image(image, angle):
  # Get the image dimensions
  height, width = image.shape[:2]
  
  # Calculate the center of rotation
  center = (width // 2, height // 2)
  
  # Compute the rotation matrix
  rotation_matrix = cv.getRotationMatrix2D(center, angle, 1.0)
  
  # Apply the rotation using warpAffine
  rotated_image = cv.warpAffine(image, rotation_matrix, (width, height), flags=cv.INTER_LINEAR)
  
  return rotated_image
 
# Load the Templete image
img = cv.imread("Templete_Input.png")
 
# Was the image there?
if img is None:
  print("Error: File not found")
  exit(0)
 
# Convert image to grayscale
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
# Convert image to binary
_, bw = cv.threshold(gray, 50, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
 
# Find all the contours in the thresholded image
contours, _ = cv.findContours(bw, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)

template_orientation = None
 
for i, c in enumerate(contours):
  # Calculate the area of each contour
  area = cv.contourArea(c)
 
  # Ignore contours that are too small or too large
  if area < 1000:
    continue
 
  # Draw each contour only for visualisation purposes
  cv.drawContours(img, contours, i, (0, 0, 255), 2)
 
  # Find the orientation of each shape
  template_orientation = getOrientation(c, img)
  
# read the input files
repository_path = "./KIMIA dataset"

output_directory = "Rotated_Images"
if not os.path.exists(output_directory):
  os.makedirs(output_directory)


for image_filename in os.listdir(repository_path):
  image_path = os.path.join(repository_path, image_filename)
  
  # Load the image
  img = cv.imread(image_path)
  
  # Convert the image to grayscale
  gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  
  # Threshold the image to create a binary image if needed
  _, bw = cv.threshold(gray, 50, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
  
  # Find contours in the binary image
  contours, _ = cv.findContours(bw, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
  
  # Calculate the orientation angle for the image
  image_orientation = None
  for i, c in enumerate(contours):
    area = cv.contourArea(c)
    if area < 1000:
      continue
    image_orientation = getOrientation(c, img)
  
  # Rotate the image to match the template orientation
  if image_orientation is not None and template_orientation is not None:
    rotation_angle = (template_orientation - image_orientation) * 180 / np.pi
    rotated_image = rotate_image(img, -rotation_angle)
    output_path = os.path.join(output_directory, "rotated_" + image_filename)
    cv.imwrite(output_path, rotated_image)

#### Algorithm
- Import necessary libraries, including OpenCV, NumPy, and os.

- Define a function getOrientation to calculate the orientation angle of a set of points.

- Define a function rotate_image to rotate an image by a given angle.

- Load a template image ("Templete_Input.png").

- Convert the loaded image to grayscale.

- Threshold the grayscale image to create a binary image.

- Find contours in the binary image.

- Loop through the detected contours:

- Calculate the area of each contour.
- Draw the contours on the original image (for visualization).
- Calculate the orientation angle using the getOrientation function.
- Read a directory of input images located in "KIMIA dataset."

- Create an output directory named "Rotated_Images" if it doesn't exist.

- Loop through each image in the input directory:

- Load the image.
- Convert the image to grayscale.
- Threshold the grayscale image to create a binary image.
- Find contours in the binary image.
- Calculate the orientation angle for the image by analyzing its contours.
- Calculate the rotation angle needed to align the image with the template orientation.
- Rotate the image using the rotate_image function.
- Save the rotated image to the "Rotated_Images" directory with a filename prefix "rotated_."

#### Flowchart of the algorithm

<img src="flowchart.jpg" alt="Task 4.1" style="border: 1px solid black; padding: 20px; background-color: white;">